# Inference with `google/gemma-7b-it`. Vanilla experiment

## Purpose
This notebook performs inference using the `google/gemma-7b-it` language model from Hugging Face's `transformers` library. The test dataset consists of 1363 records, which are fetched via an HTTP call to the **Palaven API**—a custom API developed exclusively for this thesis. The generated responses are saved back to the Palaven API for future analysis.

## Overview

1. **Declarion of global variables**:

2. **Load the model and the tokenizer**:
    - The `google/gemma-7b-it` model and its corresponding tokenizer are loaded from the Hugging Face library (`transformers`).
    - The model is used in its "out of the box" configuration, meaning no fine-tuning is applied at this stage.

3. **Inference Execution**:
    - Test dataset fetching. The test dataset, containing 1363 records, is fetched from a cloud service through the Palaven API using an HTTP request.The dataset is processed in batches, with each batch containing 50 instructions.
    - The model generates responses for each instruction in the dataset.
    - The responses are persisted back to the cloud using the Palaven API.

4. **Evaluation Metrics**:
    - **BERTScore**: Precision, Recall, and F1 scores are calculated for each batch.
    - **ROUGE**: ROUGE 1, ROUGE 2, and ROUGE L are computed, evaluating Precision, Recall, and F1.
    - **BLEU**: The BLEU score is calculated for each batch.
    - All metrics are also persisted to the cloud via the Palaven API for further analysis.

## Key Sections
### 1. Declaration of global variables

| **Variable**             | **Description**                                                                                   | **Value**                                                   |
|--------------------------|---------------------------------------------------------------------------------------------------|-------------------------------------------------------------|
| `hface_read_token`        | Access token for reading from Hugging Face, retrieved from user data.                             | `userdata.get('hface-read-token')`                          |
| `palaven_base_url`        | Base URL for the Palaven API, retrieved from user data.                                           | `userdata.get('palaven-base-url')`                          |
| `batch_size`              | Batch size for inference and evaluation execution.                                                | `50`                                                        |
| `dataset_id`              | Unique identifier for the dataset used in evaluation.                                             | `'F0444B12-5485-4299-B03B-3BDB6D4A2578'`                    |
| `evaluation_session_id`   | Unique identifier for the current evaluation session.                                             | `'EB9C5839-7B20-4D7D-B3F7-17528180676D'`                    |
| `llm_palaven_name`        | Name of the model used in Palaven.                                                 | `'google-gemma'`                                            |
| `llm_model_name`          | Full name of the model used from Hugging Face.                                                    | `'google/gemma-7b-it'`                                      |
| `device_info`             | Information about the device where the inferences and evaluations are executed.                   | `'GPU A100'`                                                |


In [2]:
from google.colab import userdata
from palaven_api_v2 import PalavenApi

hface_read_token = userdata.get('hface-read-token')
palaven_base_url = userdata.get('palaven-base-url')

batch_size = 50
dataset_id = 'F0444B12-5485-4299-B03B-3BDB6D4A2578'
evaluation_session_id = 'EB9C5839-7B20-4D7D-B3F7-17528180676D'
llm_palaven_name = 'google-gemma'
llm_model_name = 'google/gemma-7b-it'
device_info = 'GPU A100'

palaven_api = PalavenApi(palaven_base_url)

### 2. Load the model and the tokenizer

In [ ]:
!pip install -U transformers
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 100.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(llm_model_name, token=hface_read_token)
model = AutoModelForCausalLM.from_pretrained(llm_model_name, device_map="auto", token=hface_read_token).to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

### 3. Inference Execution

In [ ]:
import time
import torch

def get_chat_completion(instruction_prompt, instruction_id, instruction_count):

  start_time = time.time()

  tokenized_instruction = tokenizer(instruction_prompt, return_tensors="pt").to("cuda")

  with torch.no_grad():
    chat_completion_result = model.generate(**tokenized_instruction, max_new_tokens=1245)

  chat_completion = tokenizer.decode(chat_completion_result[0])

  end_time = time.time()
  elapsed_time = end_time - start_time

  print(f'{instruction_count} - LLM-ChatCompletion. InstructionId: {instruction_id},  Elapsed-Time: {elapsed_time:.2f} seconds')

  return chat_completion, elapsed_time

In [ ]:
def add_instruction_to_df(df, instruction_id, instruction):
    df.loc[instruction_id, 'instruction'] = instruction
    df.loc[instruction_id, 'chat_completion'] = None
    df.loc[instruction_id, 'elapsed_time'] = None

In [ ]:
import pandas as pd
import numpy as np
import json

data_shape = {
    'instruction_id': [],
    'instruction': [],
    'chat_completion': [],
    'elapsed_time': []
}

for batch_number in range(1, 29):

  print(f'Start fetching batch {batch_number}')

  instructions = palaven_api.fetch_instruction_test_dataset(evaluation_session_id, batch_number=batch_number)

  print(f'Start fetching batch done...')

  llm_responses_df = pd.DataFrame(data_shape)
  llm_responses_df.set_index('instruction_id', inplace=True)

  for item in instructions:
    add_instruction_to_df(llm_responses_df, item['instructionId'], item['instruction'])

  instruction_count = 1

  for index, row in llm_responses_df.iterrows():
    instruction = llm_responses_df.loc[index, 'instruction']

    instruction_prompt = f"""
      <start_of_turn>user
      Answer the following question in a concise and informative manner. The question is written in Spanish language, then answer in Spanish language.
      {instruction}<end_of_turn>
      <start_of_turn>model
    """

    instruction_id = int(index)

    chat_completion, elapsed_time = get_chat_completion(instruction_prompt, instruction_id, instruction_count)
    palaven_api.save_model_response(evaluation_session_id, 'llmvanilla', batch_number, instruction_id, chat_completion, elapsed_time)

    instruction_count += 1

Start fetching batch 1
Start fetching batch done...


<ipython-input-6-9ad68d3380dc>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '¿Bajo qué circunstancias los residentes en el extranjero deben pagar impuesto sobre la renta en México según el Artículo 1 de la Ley del Impuesto Sobre la Renta del 2024?' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[instruction_id, 'instruction'] = instruction


1 - LLM-ChatCompletion. InstructionId: 3878,  Elapsed-Time: 7.49 seconds
Palaven.SaveResponse. InstructionId: 3878,  Elapsed-Time: 0.92 seconds
2 - LLM-ChatCompletion. InstructionId: 3880,  Elapsed-Time: 6.21 seconds
Palaven.SaveResponse. InstructionId: 3880,  Elapsed-Time: 0.57 seconds


### 4. Evaluation metrics

### 4.1. BERTScore

In [ ]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.6 MB/s eta 0:00:00


In [5]:
import pandas as pd

def build_responses_df():
  dataframe = pd.DataFrame(columns=['instruction_id',
    'evaluation_session_id', 'dataset_id', 'batch_size', 'large_language_model',
    'device_info', 'excercise_type', 'batch_number', 'instruction',
    'response', 'category', 'response_to_evaluate', 'elapsed_time'])

  dataframe['evaluation_session_id'] = dataframe['evaluation_session_id'].astype('object')
  dataframe['dataset_id'] = dataframe['dataset_id'].astype('object')
  dataframe['large_language_model'] = dataframe['large_language_model'].astype('object')
  dataframe['device_info'] = dataframe['device_info'].astype('object')
  dataframe['excercise_type'] = dataframe['excercise_type'].astype('object')
  dataframe['instruction'] = dataframe['instruction'].astype('object')
  dataframe['response'] = dataframe['response'].astype('object')
  dataframe['category'] = dataframe['category'].astype('object')
  dataframe['response_to_evaluate'] = dataframe['response_to_evaluate'].astype('object')

  dataframe.set_index('instruction_id', inplace=True)

  return dataframe


def add_response_to_df(dataframe, response):
  dataframe.loc[response['instructionId'], 'evaluation_session_id'] = response['evaluationSessionId']
  dataframe.loc[response['instructionId'], 'dataset_id'] = response['datasetId']
  dataframe.loc[response['instructionId'], 'batch_size'] = response['batchSize']
  dataframe.loc[response['instructionId'], 'large_language_model'] = response['largeLanguageModel']
  dataframe.loc[response['instructionId'], 'device_info'] = response['deviceInfo']
  dataframe.loc[response['instructionId'], 'excercise_type'] = response['evaluationExercise']
  dataframe.loc[response['instructionId'], 'batch_number'] = response['batchNumber']
  dataframe.loc[response['instructionId'], 'instruction'] = response['instruction']
  dataframe.loc[response['instructionId'], 'response'] = response['response']
  dataframe.loc[response['instructionId'], 'category'] = response['category']
  dataframe.loc[response['instructionId'], 'response_to_evaluate'] = response['llmResponseToEvaluate']
  dataframe.loc[response['instructionId'], 'elapsed_time'] = response['elapsedTime']

In [ ]:
from bert_score import score as bert_score

for batch_number in range(1, 29):

  model_responses_df = build_responses_df()

  print(f'Palaven. Bertscore evaluation. Start processing batch: {batch_number}')

  model_responses = palaven_api.fetch_model_responses(evaluation_session_id, 'llmvanilla', batch_number)

  for item in model_responses:
    add_response_to_df(model_responses_df, item)

  references = model_responses_df['response'].tolist()
  candidates = model_responses_df['response_to_evaluate'].tolist()

  accuracy, recall, f1 = bert_score(candidates, references, lang='es', verbose=True, device='cuda')

  accuracy = accuracy.tolist()
  recall = recall.tolist()
  f1 = f1.tolist()

  average_accuracy = sum(accuracy) / len(accuracy)
  average_recall = sum(recall) / len(recall)
  average_f1 = sum(f1) / len(f1)

  print(f'Palaven. Bertscore evaluation. Batch: {batch_number}. Average accuracy: {average_accuracy}. Average recall: {average_recall}. Average F1: {average_f1}')

  palaven_api.save_bert_score_metrics(evaluation_session_id, 'llmvanilla', batch_number, average_accuracy, average_recall, average_f1)

  print(f'Palaven. Bertscore evaluation. Posted batch {batch_number}')

Palaven. Bertscore evaluation. Start processing batch: 1
calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.62 seconds, 80.68 sentences/sec
Palaven. Bertscore evaluation. Batch: 1. Average accuracy: 0.6518882083892822. Average recall: 0.7236301374435424. Average F1: 0.6849419367313385
Palaven. Bertscore evaluation. Posted batch 1
Palaven. Bertscore evaluation. Start processing batch: 2
calculating scores...
computing bert embedding.


...

done in 0.57 seconds, 87.14 sentences/sec
Palaven. Bertscore evaluation. Batch: 27. Average accuracy: 0.6664226853847504. Average recall: 0.7349530589580536. Average F1: 0.6981767296791077
Palaven. Bertscore evaluation. Posted batch 27
Palaven. Bertscore evaluation. Start processing batch: 28
calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.23 seconds, 56.64 sentences/sec
Palaven. Bertscore evaluation. Batch: 28. Average accuracy: 0.6898812468235309. Average recall: 0.7781575597249545. Average F1: 0.7305226784486037
Palaven. Bertscore evaluation. Posted batch 28


### 4.2. ROUGE (1,2 and L)

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=bf4ee252c2968c85657c52bbfd4f30090c482508e655ef8cbf4dff97685796d2
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from rouge_score import rouge_scorer

for batch_number in range(1, 29):

  model_responses_df = build_responses_df()

  print(f'Palaven. ROUGE evaluation. Start processing batch: {batch_number}')
  model_responses = palaven_api.fetch_model_responses(evaluation_session_id, 'llmvanilla', batch_number)

  for item in model_responses:
    add_response_to_df(model_responses_df, item)

  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

  r1_precision, r1_recall, r1_f1 = [], [], []
  r2_precision, r2_recall, r2_f1 = [], [], []
  rL_precision, rL_recall, rL_f1 = [], [], []

  for _, row in model_responses_df.iterrows():
    reference = row['response']
    candidate = row['response_to_evaluate']

    scores = scorer.score(reference, candidate)

    #ROUGE-1
    r1_precision.append(scores['rouge1'].precision)
    r1_recall.append(scores['rouge1'].recall)
    r1_f1.append(scores['rouge1'].fmeasure)

    #ROUGE-2
    r2_precision.append(scores['rouge2'].precision)
    r2_recall.append(scores['rouge2'].recall)
    r2_f1.append(scores['rouge2'].fmeasure)

    #ROUGE-L
    rL_precision.append(scores['rougeL'].precision)
    rL_recall.append(scores['rougeL'].recall)
    rL_f1.append(scores['rougeL'].fmeasure)


  average_r1_precision = sum(r1_precision) / len(r1_precision)
  average_r1_recall = sum(r1_recall) / len(r1_recall)
  average_r1_f1 = sum(r1_f1) / len(r1_f1)

  average_r2_precision = sum(r2_precision) / len(r2_precision)
  average_r2_recall = sum(r2_recall) / len(r2_recall)
  average_r2_f1 = sum(r2_f1) / len(r2_f1)

  average_rL_precision = sum(rL_precision) / len(rL_precision)
  average_rL_recall = sum(rL_recall) / len(rL_recall)
  average_rL_f1 = sum(rL_f1) / len(rL_f1)

  rouge_metrics = []

  rouge_metrics.append({
      'rougeScoreType': 'rouge1',
      'batchNumber': batch_number,
      'precision': average_r1_precision,
      'recall': average_r1_recall,
      'f1': average_r1_f1
  });

  rouge_metrics.append({
      'rougeScoreType': 'rouge2',
      'batchNumber': batch_number,
      'precision': average_r2_precision,
      'recall': average_r2_recall,
      'f1': average_r2_f1
  });

  rouge_metrics.append({
      'rougeScoreType': 'rougeL',
      'batchNumber': batch_number,
      'precision': average_rL_precision,
      'recall': average_rL_recall,
      'f1': average_rL_f1
  });

  print(rouge_metrics)

  print(f'Palaven. ROUGE evaluation finished for batch: {batch_number}.')

  palaven_api.save_rouge_metrics(evaluation_session_id, 'llmvanilla', batch_number, rouge_metrics)

  print(f'Palaven. Bertscore evaluation. Posted batch {batch_number}')

Palaven. ROUGE evaluation. Start processing batch: 1
[{'rougeScoreType': 'rouge1', 'batchNumber': 1, 'precision': 0.1693740802677117, 'recall': 0.58109845672844, 'f1': 0.23611279172686714}, {'rougeScoreType': 'rouge2', 'batchNumber': 1, 'precision': 0.07865558074992762, 'recall': 0.23509923410950187, 'f1': 0.10803429454356323}, {'rougeScoreType': 'rougeL', 'batchNumber': 1, 'precision': 0.12425966378766576, 'recall': 0.45676317229663, 'f1': 0.17554089831192654}]
Palaven. ROUGE evaluation finished for batch: 1.
Palaven. Bertscore evaluation. Posted batch 1
Palaven. ROUGE evaluation. Start processing batch: 2
[{'rougeScoreType': 'rouge1', 'batchNumber': 2, 'precision': 0.23116246397225607, 'recall': 0.5093811767023823, 'f1': 0.28289991903115835}, {'rougeScoreType': 'rouge2', 'batchNumber': 2, 'precision': 0.09607454370612736, 'recall': 0.22925890048775646, 'f1': 0.12104588902217653}, {'rougeScoreType': 'rougeL', 'batchNumber': 2, 'precision': 0.166653205001818, 'recall': 0.38348338326833

### 4.3. BLEU

In [7]:
from nltk.translate.bleu_score import sentence_bleu

for batch_number in range(1, 29):

    model_responses_df = build_responses_df()

    print(f'Palaven. BLEU evaluation. Start processing batch: {batch_number}')

    model_responses = palaven_api.fetch_model_responses(evaluation_session_id, 'llmvanilla', batch_number)

    for item in model_responses:
        add_response_to_df(model_responses_df, item)

    references = model_responses_df['response'].tolist()
    candidates = model_responses_df['response_to_evaluate'].tolist()

    bleu_scores = []

    for candidate, reference in zip(candidates, references):
        reference_tokens = [reference.split()]
        candidate_tokens = candidate.split()
        bleu_score = sentence_bleu(reference_tokens, candidate_tokens)
        bleu_scores.append(bleu_score)

    average_bleu = sum(bleu_scores) / len(bleu_scores)

    print(f'Palaven. BLEU evaluation. Batch: {batch_number}. Average BLEU score: {average_bleu}')

    palaven_api.save_bleu_score_metrics(evaluation_session_id, 'llmvanilla', batch_number, average_bleu)

    print(f'Palaven. BLEU evaluation. Posted batch {batch_number}')

Palaven. BLEU evaluation. Start processing batch: 1


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Palaven. BLEU evaluation. Batch: 1. Average BLEU score: 0.03960964630838596
Palaven. BLEU evaluation. Posted batch 1
Palaven. BLEU evaluation. Start processing batch: 2


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Palaven. BLEU evaluation. Batch: 2. Average BLEU score: 0.05040291265226397
Palaven. BLEU evaluation. Posted batch 2
Palaven. BLEU evaluation. Start processing batch: 3


...

Palaven. BLEU evaluation. Batch: 26. Average BLEU score: 0.03583940436011648
Palaven. BLEU evaluation. Posted batch 26
Palaven. BLEU evaluation. Start processing batch: 27


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Palaven. BLEU evaluation. Batch: 27. Average BLEU score: 0.04504065267431197
Palaven. BLEU evaluation. Posted batch 27
Palaven. BLEU evaluation. Start processing batch: 28


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Palaven. BLEU evaluation. Batch: 28. Average BLEU score: 0.08600271170371249
Palaven. BLEU evaluation. Posted batch 28
